Install required Libraries

In [ ]:
# Model
!pip install -q transformers accelerate einops langchain bitsandbytes

# For API
!pip -q install fastapi
!pip -q install pyngrok
!pip -q install uvicorn
!pip -q install nest_asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, pipeline

import torch

model_name = "tiiuae/falcon-7b-instruct" #tiiuae/falcon-40b-instruct
model = AutoModelForCausalLM.from_pretrained(model_name, load_in_8bit=True, device_map="auto")
model = torch.compile(model)

tokenizer = AutoTokenizer.from_pretrained(model_name)

pipeline = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    trust_remote_code=True,
    device_map="auto",
    max_length=512,
    do_sample=False,
    top_k=10,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
)

A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- configuration_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/tiiuae/falcon-7b-instruct:
- modelling_RW.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


Craft Prompt Tempelate and wrap model and template in LangChain LLM Chain

In [ ]:
from langchain import PromptTemplate, LLMChain

falcon = HuggingFacePipeline(pipeline=pipeline, model_kwargs={"temperature": 0.0})

def answer_with_falcon(riddle):
  template = """
      You are a science prodigy currently competing in a National Science competition. You are now in the fifth round, where you must provide a one-word answer to a riddle. Remember, your answer should consist of just the term the riddle is pointing to, and nothing else. Adding additional text will result in point deductions.
      Here's an example to guide you:
      Riddle: you might think i am a rather unstable character because i never stay at one place, however my motion obeys strict rules and i always return to where i started and even if i have to leave that spot again i do it in strict accordance to time, i can be named in electrical and mechanical contexts in all cases i obey the same mathematical rules, in order to fully analyse me you would think about a stiffness or force constant restoring force and angular frequency,
      Answer: oscillator

      Read the riddle below and provide the correct answer.
      Riddle: {riddle}

      Answer:"""

  prompt = PromptTemplate(template=template, input_variables=["riddle"])

  falcon_chain = LLMChain(prompt=prompt, llm=falcon)

  return falcon_chain.run(riddle)

Core code needed to set up API

In [ ]:
import uvicorn
import fastapi
from pyngrok import ngrok
from pydantic import BaseModel
import nest_asyncio

nest_asyncio.apply()

In [ ]:
class InputText(BaseModel):
  text: str

class OutputText(BaseModel):
  answer: str

In [ ]:
!ngrok config add-authtoken <YOUR AUTH TOKEN>
!ngrok diagnose

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
Testing ngrok connectivity...

Internet Connectivity
  Name Resolution                           [ OK ]
  TCP                                       [ OK ]
  TLS                                       [ OK ]
Localhost Connectivity
  Name Resolution                           [ OK ]
Ngrok Connectivity - Region: United States
  Name Resolution                           [ OK ]
  TCP                                       [ OK ]
  TLS                                       [ OK ]
  Tunnel Protocol                           [ OK ]
Successfully established ngrok connection! (region: 'us', latency: 95.312806ms)



In [ ]:
app = fastapi.FastAPI()

@app.get("/answer", response_model=OutputText)
def answer(input_text: InputText):
  gen_output = answer_with_falcon(input_text.text)
  return {"answer": gen_output}

@app.get("/qa-test", response_model=OutputText)
async def qa_test():
    return {"answer": "Hello from QA"}

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print("Public URL:", ngrok_tunnel.public_url)
uvicorn.run(app, port=8000)

INFO:     Started server process [977]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://3e06-35-204-173-228.ngrok-free.app
INFO:     47.14.177.179:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     47.14.177.179:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     47.14.177.179:0 - "GET /qa-test HTTP/1.1" 200 OK
INFO:     47.14.177.179:0 - "GET /qa-test HTTP/1.1" 200 OK


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


INFO:     47.14.177.179:0 - "GET /answer HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


INFO:     47.14.177.179:0 - "GET /answer HTTP/1.1" 200 OK


Setting `pad_token_id` to `eos_token_id`:11 for open-end generation.


INFO:     47.14.177.179:0 - "GET /answer HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [977]
